# Lookahead Decoding with LLAMA 7B
In this tutorial, you will use LMI container from DLC to SageMaker and run inference with it.

Please make sure the following permission granted before running the notebook:

- S3 bucket push access
- SageMaker access

## Step 1: Let's bump up SageMaker and import stuff

In [ ]:
%pip install sagemaker --upgrade  --quiet

In [ ]:
import boto3
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

## Step 2: Start preparing model artifacts
In LMI contianer, we expect some artifacts to help setting up the model
- serving.properties (required): Defines the model server settings
- model.py (optional): A python file to define the core inference logic
- requirements.txt (optional): Any additional pip wheel need to install

In [ ]:
%%writefile serving.properties
engine=Python
option.model_id=daryl149/llama-2-7b-chat-hf
option.task=text-generation
option.tensor_parallel_degree=1
option.entryPoint=model.py
option.device_map=auto
option.dtype=fp16

In [ ]:
%%writefile requirements.txt
transformers==4.34.0
accelerate==0.23.0
git+https://github.com/nd7141/LookaheadDecoding.git

In this step, we will try to override the [default HuggingFace handler](https://github.com/deepjavalibrary/djl-serving/blob/0.25.0-dlc/engines/python/setup/djl_python/huggingface.py) provided by DJLServing. We will add an extra parameter checker called `password` to see if password is correct in the payload.

In [ ]:
%%writefile model.py
from djl_python.huggingface import HuggingFaceService
from djl_python import Output
from djl_python.encode_decode import encode, decode
import logging
import json
import types
import os 
os.environ["USE_LADE"]="1"
import lade
lade.augment_all()
lade.config_lade(LEVEL=5, WINDOW_SIZE=7, GUESS_SET_SIZE=7, DEBUG=0)

_service = HuggingFaceService()

def handle(inputs):
    if not _service.initialized:
        _service.initialize(inputs.get_properties())

    if inputs.is_empty():
        # initialization request
        return None

    return _service.inference(inputs)

In [ ]:
%%sh
rm -rf mymodel
mkdir mymodel
cp serving.properties mymodel/
cp model.py mymodel/
cp requirements.txt mymodel/
tar czvf mymodel.tar.gz mymodel/


## Step 3: Start building SageMaker endpoint
In this step, we will build SageMaker endpoint from scratch

### Getting the container image URI

[Large Model Inference available DLC](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#large-model-inference-containers)


In [ ]:
image_uri = image_uris.retrieve(
        framework="djl-deepspeed",
        region=sess.boto_session.region_name,
        version="0.25.0"
    )

### Upload artifact on S3 and create SageMaker model

In [ ]:
s3_code_prefix = "large-model-lmi/code"
bucket = sess.default_bucket()  # bucket to house artifacts
code_artifact = sess.upload_data("mymodel.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")

model = Model(image_uri=image_uri, model_data=code_artifact, role=role)

### 4.2 Create SageMaker endpoint

You need to specify the instance to use and endpoint names

In [ ]:
import time 
instance_type = "ml.g5.2xlarge"
endpoint_name = sagemaker.utils.name_from_base("lmi-model-lade")
print('Endpoint name:', endpoint_name)

start = time.time()
model.deploy(initial_instance_count=1,
             instance_type=instance_type,
             endpoint_name=endpoint_name,
             container_startup_health_check_timeout=1800
            )
finish = time.time()
print(f'Deployment time: {finish-start:.2f} sec')

In [ ]:
# our requests and responses will be in json format so we specify the serializer and the deserializer
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=serializers.JSONSerializer(),
)

## Step 5: Test and benchmark the inference

Firstly let's try to run with a wrong inputs

In [ ]:
import time 
start = time.time()
results = predictor.predict(
    {"inputs": "Write 1000 words text about risks of AI", "parameters": {"max_new_tokens":1024, "do_sample": False}}
)
finish = time.time()
print(f'Time {finish-start:.2f}')

Then let's run with the right one

In [ ]:
results

In [ ]:
predictor.predict(
    {"inputs": "Large model inference is", "parameters": {}, "password": "12345"}
)

## Clean up the environment

In [ ]:
sess.delete_endpoint(endpoint_name)
sess.delete_endpoint_config(endpoint_name)
model.delete_model()